In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as sc
from operator import mul
import statistics as stats
import math

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS, Isomap, LocallyLinearEmbedding

In [ ]:
#Changement des virgules en points pour obtenir des valeurs numériques
text = open("shealth.csv", 'r')
text = ''.join([i for i in text]).replace(',', '.')
x = open("output.csv",'w')
x.writelines(text)
x.close()

In [ ]:
shealth = pd.read_csv("output.csv", sep=';')

In [ ]:
#Modification des temps de style 8h en 8h00
for k in range(608):
    if type(shealth['sleep time'][k]) == type('hello'):
        hdata = shealth['sleep time'][k].split('h')
        if hdata[1] == '':
            hdata[1] = '00'
            shealth['sleep time'][k] = 'h'.join(hdata)

In [ ]:
#Copie du jeu de données pour remplacer les valeurs de temps de sommeil
shealth_modif = shealth.copy(deep=True)
shealth_modif['sleep time'] = shealth_modif['sleep time'].replace(np.nan, 'nan', regex=True)

In [ ]:
#Modification de l'heure en minute en valeur numérique
factors = (60, 1)
for k in range(608):
    sleep_min = 0
    if shealth_modif['sleep time'][k] != 'nan':
        sleep_min = sum(i*j for i, j in zip(map(int, shealth['sleep time'][k].split('h')), factors))
        shealth_modif['sleep time'] = shealth_modif['sleep time'].replace(shealth_modif['sleep time'][k], sleep_min)

In [ ]:
shealth_modif['sleep time'] = shealth_modif['sleep time'].replace('nan', np.nan, regex=True)

In [ ]:
plt.plot(shealth_modif['Date'], shealth_modif['Weigth'])
plt.xlabel('Date')
plt.ylabel('Poids')
plt.title('Poids en fonction de la date')

In [ ]:
plt.plot(shealth_modif['Date'], shealth_modif['Steps'])
plt.xlabel('Date')
plt.ylabel('Nombre de pas')
plt.title('Nombre de pas en fonction de la date')

In [ ]:
plt.plot(shealth_modif['Date'], shealth_modif['Walk (min)'])
plt.xlabel('Date')
plt.ylabel('Temps de marche')
plt.title('Temps de marche en fonction de la date')

In [ ]:
plt.plot(shealth_modif['Date'], shealth_modif['Sport (min)'])
plt.xlabel('Date')
plt.ylabel('Temps de sport')
plt.title('Temps de sport en fonction de la date')

In [ ]:
plt.plot(shealth_modif['Date'], shealth_modif['Spent kcal'])
plt.xlabel('Date')
plt.ylabel('kcal dépensées')
plt.title('kcal dépensées en fonction de la date')

In [ ]:
plt.plot(shealth_modif['Date'], shealth_modif['sleep time'])
plt.xlabel('Date')
plt.ylabel('Temps de sommeil')
plt.title('Temps de sommeil en fonction de la date')

In [ ]:
plt.plot(shealth_modif['Date'], shealth_modif['GPS country'])
plt.xlabel('Date')
plt.ylabel('GPS du pays')
plt.title('GPS du pays en fonction de la date')

In [ ]:
print(shealth_modif.corr())

In [ ]:
sns.pairplot(shealth_modif)

In [ ]:
sns.heatmap(shealth_modif.corr(), annot=True)

In [ ]:
print("Intervalle de confiance Spent kcal - Walk (min)")
Z = (np.log(1+shealth_modif["Spent kcal"].corr(shealth_modif["Walk (min)"]))-np.log(1-shealth_modif["Spent kcal"].corr(shealth_modif["Walk (min)"])))/2
s=np.sqrt(1/(147))
Zinf=Z-1.96*s
Zsup=Z+1.96*s
print('[', ((np.exp(2*Zinf))-1)/((np.exp(2*Zinf))+1), ' ; ', ((np.exp(2*Zsup))-1)/((np.exp(2*Zsup))+1), "]")

In [ ]:
sc.chi2_contingency(pd.crosstab(shealth_modif["Spent kcal"], shealth_modif["Walk (min)"]))

In [ ]:
for i in range(10):
    shealth_modif['Sport (min)'] = shealth_modif['Sport (min)'].replace(i, np.nan, regex=True)

In [ ]:
plt.scatter(shealth_modif['Sport (min)'], shealth_modif['Weigth'])
plt.xlabel('Temps de sport')
plt.ylabel('Poids')
plt.title('Poids en fonction du temps de sport')

In [ ]:
diff_weigth = np.array([])
for i in range(608):
    diff = shealth_modif['Weigth'][i+1] - shealth_modif['Weigth'][i]
    diff_weigth = np.append(diff_weigth, diff)

In [ ]:
#plt.plot(shealth_modif['Date'][:608][::12], diff_weigth[::12])
plt.plot(shealth_modif['Date'][:608], diff_weigth)
plt.xlabel('Date')
plt.ylabel('Différence du poids')
plt.title('Différence du poids en fonction de la date')

In [ ]:
plt.scatter(shealth_modif['Sport (min)'][:608], diff_weigth)
plt.xlabel('Temps de sport')
plt.ylabel('Différence du poids')
plt.title('Différence du poids en fonction du temps de sport')

In [ ]:
mean_weigth = np.array([])
for i in range(212, 577, 7):
    mean = diff_weigth[i:i+7].sum()/7
    mean_weigth = np.append(mean_weigth, mean)

In [ ]:
mean_walk = np.array([])
for i in range(212, 577, 7):
    mean = shealth_modif['Walk (min)'][i:i+7].sum()/7
    mean_walk = np.append(mean_walk, mean)

In [ ]:
plt.plot(shealth_modif['Date'][212:570][::7], mean_weigth[:-1])
plt.xlabel('Date')
plt.ylabel('Moyenne du poids sur une semaine')
plt.title('Moyenne du poids en fonction de la date pour l\'année complète 2016')

In [ ]:
sum_steps = np.array([])
for i in range(7):
    sum_val = 0
    for j in range(0, 608, 7):
        if not np.isnan(shealth_modif['Steps'][i+j]):
            sum_val += shealth_modif['Steps'][i+j]
    sum_steps = np.append(sum_steps, sum_val)

In [ ]:
steps_order = np.delete(sum_steps, [5, 6])
steps_order = np.insert(steps_order, 0, sum_steps[5])
steps_order = np.insert(steps_order, 1, sum_steps[6])

In [ ]:
plt.bar(['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'], steps_order)
plt.xlabel('Jour')
plt.ylabel('Total de pas')
plt.title('Total de pas sur la période en fonction du jour')

In [ ]:
sum_sport = np.array([])
for i in range(7):
    sum_val = 0
    for j in range(0, 608, 7):
        if not np.isnan(shealth_modif['Sport (min)'][i+j]):
            sum_val += shealth_modif['Sport (min)'][i+j]
    sum_sport = np.append(sum_sport, sum_val)

In [ ]:
sport_order = np.delete(sum_sport, [5, 6])
sport_order = np.insert(sport_order, 0, sum_sport[5])
sport_order = np.insert(sport_order, 1, sum_sport[6])

In [ ]:
plt.bar(['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'], sport_order)
plt.xlabel('Jour')
plt.ylabel('Total de temps de sport')
plt.title('Total de temps de sport sur la période en fonction du jour')

In [ ]:
sum_walk = np.array([])
for i in range(7):
    sum_val = 0
    for j in range(0, 608, 7):
        if not np.isnan(shealth_modif['Walk (min)'][i+j]):
            sum_val += shealth_modif['Walk (min)'][i+j]
    sum_walk = np.append(sum_walk, sum_val)

In [ ]:
walk_order = np.delete(sum_walk, [5, 6])
walk_order = np.insert(walk_order, 0, sum_walk[5])
walk_order = np.insert(walk_order, 1, sum_walk[6])

In [ ]:
plt.bar(['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'], walk_order)
plt.xlabel('Jour')
plt.ylabel('Total de temps de marche')
plt.title('Total de temps de marche sur la période en fonction du jour')

In [ ]:
#Création et différenciation de Weight avec ajout de colonne loss
def diff_weight(weight_list): 
    loss = [] #0 si perte de poids et 1 sinon
    diff_weight=[]
    ex_loss = []
    ex_diff_weight = []
    for i in range(weight_list.size):
        loss.append(0) #Dernier élément qui vaut 0
        diff_weight.append(None)
    for i in range(0,weight_list.size-1):
        if (weight_list[i] and weight_list[i+1]):
            diff_weight[i] = (weight_list[i+1]-weight_list[i])
            if not diff_weight[i]==None:
                if diff_weight[i] >= 0:
                    loss[i] = 1
                else:
                    loss[i] = 0
    for j in range(len(diff_weight)):
        if not diff_weight[j] == None and not np.isnan(diff_weight[j]):
            if not loss[j] == None:
                ex_loss.append(loss[j])
                ex_diff_weight.append(diff_weight[j])

    df['diff_weight']=diff_weight
    df['loss']=loss
    return(ex_loss)

In [ ]:
#Génération à chaque fois d'un array résumant chaque semaine:
    #poids: diff
    #steps, kcal, sport: addition de tout sur 7 jours
def diff_arrays(data):
    #initializing
    extracted_date = []
    extracted_data =[]
    data_list=df[data]
    day_list=df['Date']
    weeks_data = []
    weeks_date = []
    weeks_data.insert(0,None)
    weeks_date.insert(0,None)
    #creating full length arrays
    for i in range(1,data_list.size):
        if i%7 ==0 and data == 'Weigth':
            weeks_date.append(day_list[i])
            weeks_data.append(data_list[i-7]-data_list[i])
        elif (i%7 ==0 and not data == 'Weigth'):
            weeks_date.append(day_list[i])
            weeks_data.append(data_list[i-7]+data_list[i-6]+data_list[i-5]+data_list[i-4]+data_list[i-3]+data_list[i-2]+data_list[i-1]+data_list[i])
        else:
            weeks_date.append(None)
            weeks_data.append(None)
    #xtracting non None values
    for j in range(len(weeks_data)):
        if not weeks_data[j] == None:
            if not weeks_date[j] == None:
                extracted_data.append(weeks_data[j])
                extracted_date.append(weeks_date[j])
    return(extracted_data)

In [ ]:
#plot hists avec courbe tendance
def plot_hist(data_list):
    for item in data_list:
        plt.figure()
        data=item
        sns.distplot(diff_arrays(data))
        plt.title(data)
        plt.plot()

In [ ]:
#Couleur en fonction de prise/perte
def define_color1(i):
    if df.loss[i] == 1:
        return 'firebrick'
    if df.loss[i] == 0:
        return 'pink'

In [ ]:
#Couleur en fonction du jour de la semaine
def define_color(i):
    
    if df.Day[i] == 'M':
        return 'firebrick'
    if df.Day[i] == 'Tu':
        return 'blue'
    if df.Day[i] == 'W':
        return 'green'
    if df.Day[i] == 'Th':
        return 'tomato'
    if df.Day[i] == 'F':
        return 'yellow'
    if df.Day[i] == 'Sa':
        return 'grey'
    if df.Day[i] == 'Su':
        return 'black'

In [ ]:
#Clustering pca
def proj_pca():
    iris_data = scaler.fit_transform(df[liste_attributs_])
    iris_data = pd.DataFrame(iris_data)
    pca = PCA(n_components=2)
    data_proj = pca.fit_transform(iris_data)
    print('Pourcentage de variance expliquée', pca.explained_variance_ratio_)
    print(data_proj[0, 0])
    for i in range(len(df['loss'])): 
        plt.scatter(data_proj[i][0], data_proj[i][1], color=define_color1(i))
    perte = mpatches.Patch(color='firebrick', label='prise')
    prise = mpatches.Patch(color='pink', label='perte')
    plt.legend(handles=[perte, prise])
    plt.title('Représentation du dataset')  
    plt.show()

In [ ]:
def proj_lle():
    data = scaler.fit_transform(df[liste_attributs_])

    #LLE
    data_proj_LLE3 = LocallyLinearEmbedding(n_neighbors=3).fit_transform(data)
    data_proj_LLE5 = LocallyLinearEmbedding(n_neighbors=5).fit_transform(data)
    data_proj_LLE8 = LocallyLinearEmbedding(n_neighbors=8).fit_transform(data)
    data_proj_LLE10 = LocallyLinearEmbedding(n_neighbors=10).fit_transform(data)
    data_proj_LLE12 =  LocallyLinearEmbedding(n_neighbors=12).fit_transform(data)
    data_proj_LLE15 = LocallyLinearEmbedding(n_neighbors=15).fit_transform(data)
    
    
    fig2, axs = plt.subplots(2, 3)
    fig2.tight_layout()
    
    for i in range(72):
        axs[0][0].scatter(data_proj_LLE3[i][0], data_proj_LLE3[i][1], color=define_color(i))
        axs[0][0].set_title('Voisins : 3')
        axs[1][0].scatter(data_proj_LLE5[i][0], data_proj_LLE5[i][1], color=define_color(i))
        axs[1][0].set_title('Voisins : 5')
        axs[0][1].scatter(data_proj_LLE8[i][0], data_proj_LLE8[i][1], color=define_color(i))
        axs[0][1].set_title('Voisins : 8')
        axs[1][1].scatter(data_proj_LLE10[i][0], data_proj_LLE10[i][1], color=define_color(i))
        axs[1][1].set_title('Voisins : 10')
        axs[0][2].scatter(data_proj_LLE12[i][0], data_proj_LLE12[i][1], color=define_color(i))
        axs[0][2].set_title('Voisins : 12')
        axs[1][2].scatter(data_proj_LLE15[i][0], data_proj_LLE15[i][1], color=define_color(i))
        axs[1][2].set_title('Voisins : 15')
    plt.show()

In [ ]:
df = pd.read_csv('output.csv',sep=";")
df = df[df.Weigth.notnull()] #supp les lignes sans weight
df = df[df.Steps.notnull()] #supp les lignes sans steps
df = df.reset_index(drop=True)

In [ ]:
df = shealth_modif.copy(deep=True)
df = df[df['Steps'].notnull()] #supp les lignes sans steps
df = df.reset_index(drop=True)
plt.figure()
sns.distplot(df['Steps'])
plt.title('Steps')
plt.plot()

In [ ]:
plot_hist(['Steps', 'Weigth', 'Walk (min)', 'Sport (min)', 'Spent kcal'])   
diff_weight(df['Weigth'])
liste_attributs_=[ 'Steps', 'Walk (min)', 'Sport (min)', 'Spent kcal', 'sleep time', 'GPS country'] 
liste_attributs_=['Steps', 'Spent kcal', 'Walk (min)', 'Sport (min)'] 
scaler = StandardScaler()
plt.figure()
proj_pca()  
proj_lle()

In [ ]:
def display_circles(labs):
    data = scaler.fit_transform(df[labs])
    data = pd.DataFrame(data)
    pca = PCA(n_components=2)
    proj = pca.fit_transform(data)
    pcs = pca.components_
    n_comp = 2
    axis_ranks = [(0, 1), (2, 3), (4, 5)]
    labels=np.array(labs)
    label_rotation=0
    lims=None
    for d1, d2 in axis_ranks:  # On affiche les 3 premiers plans factoriels, donc les 6 premières composantes
        if d2 < n_comp:

            # On initialise la figure
            fig, ax = plt.subplots(figsize=(7, 6))

            # On détermine les axes
            if lims is not None:
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30:
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else:
                xmin, xmax, ymin, ymax = min(pcs[d1, :]), max(pcs[d1, :]), min(pcs[d2, :]), max(pcs[d2, :])

            # On affiche les flèches
            if pcs.shape[1] < 30:
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]),
                           pcs[d1, :], pcs[d2, :],
                           angles='xy', scale_units='xy', scale=1, color="grey")
            else:
                lines = [[[0, 0], [x, y]] for x, y in pcs[[d1, d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))

            # On affiche les labels
            if labels is not None:
                for i, (x, y) in enumerate(pcs[[d1, d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax:
                        plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation,
                                 color="blue", alpha=0.5)

            # On affiche le cercle
            circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor='b')
            plt.gca().add_artist(circle)

            # On définit les limites
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
            


            # On affiche le repère dans lequel on se place
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # On nomme les axes
            plt.xlabel('F{} ({}%)'.format(d1 + 1, round(100 * pca.explained_variance_ratio_[d1], 1)))
            plt.ylabel('F{} ({}%)'.format(d2 + 1, round(100 * pca.explained_variance_ratio_[d2], 1)))

            plt.title("Cercle des corrélations (F{} et F{})".format(d1 + 1, d2 + 1))
            plt.show()


display_circles(['Steps', 'Spent kcal', 'Walk (min)', 'Sport (min)', 'Weigth'])

In [ ]:
shapiro_weight = sc.shapiro(diff_weight(df['Weigth']))
print(shapiro_weight)
print(acorr_ljungbox(diff_weight(df['Weigth']), return_df=True))

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(diff_weight(df['Weigth']), lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(diff_weight(df['Weigth']), lags=20, ax=ax2)

In [ ]:
print(adfuller(diff_weight(df['Weigth'])))
print(kpss(diff_weight(df['Weigth'])))

In [ ]:
def ch_cluster(data, cluster):
    kmeans = KMeans(n_clusters=cluster)
    prediction = kmeans.fit_predict(data)
    score = metrics.calinski_harabasz_score(data, prediction)
    #print('calinski harabsz', score)
    return score

def db_cluster(data, cluster):
    kmeans = KMeans(n_clusters=cluster)
    prediction = kmeans.fit_predict(data)
    score = metrics.davies_bouldin_score(data, prediction)
    #print('davies bouldin', score)
    return score

def plot_indexes(df):
    #calinski harabasz
    scores = []
    clusters = list(range(2,10))
    for cluster in clusters:
        scores.append(ch_cluster(df, cluster))
    print("L'indice de Calinski Harabasz est", max(scores))
    plt.figure()
    plt.plot(clusters, scores, linestyle='--', marker='x', color='r');
    plt.xlabel('Nombre de clusters');
    plt.ylabel('Indice de Calinski-Harabasz');
    plt.title('Indice de Calinski-Harabasz en fonction du nombre de clusters')
    plt.figure()
    #davies bouldin
    scores = []
    clusters = list(range(2,10))
    for cluster in clusters:
        scores.append(db_cluster(df, cluster))
    print("L'indice de Davies Bouldin est", max(scores))
    plt.plot(clusters, scores, linestyle='--', marker='x', color='b');
    plt.xlabel('Nombre de clusters');
    plt.ylabel('Indice de Davies-Bouldin');
    plt.title('Indice de Davies-Bouldin en fonction du nombre de clusters');
    #indice de silhouette
    pca = PCA(n_components=2)
    pca_val = pca.fit_transform(df)
    df_pca = pd.DataFrame(data = pca_val, columns = ['PC1', 'PC2'])
    kmeans = KMeans(n_clusters=3, n_init=5, max_iter=300, random_state=100).fit(df)
    prediction = kmeans.predict(df)
    print("L'indice de silhouette est", silhouette_score(df_pca, prediction))


def means():
    liste=['Weigth', 'Steps', 'Spent kcal', 'Walk (min)', 'Sport (min)'] 
    for item in liste:
        Zs=stats.mean(df[item])+1.96*(stats.stdev(df[item])/math.sqrt(len(df[item])))
        Zi=stats.mean(df[item])-1.96*(stats.stdev(df[item])/math.sqrt(len(df[item])))
        print('La moyenne de la liste des ', item, ' est ',stats.mean(df[item]))
        print('Son intervalle de confiance à 95% est [',Zi, ';',Zs,']' )
    dfitem = df['diff_weight'][:-1]
    Zs=stats.mean(dfitem)+1.96*(stats.stdev(dfitem)/math.sqrt(len(dfitem)))
    Zi=stats.mean(dfitem)-1.96*(stats.stdev(dfitem)/math.sqrt(len(dfitem)))
    print('La moyenne de la liste des poids différenciées est ',stats.mean(dfitem))
    print('Son intervalle de confiance à 95% est [',Zi, ';',Zs,']' )

plot_indexes(df.drop(columns=['Day','Date', 'sleep time', 'Weigth'])[:574])
means()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns;
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm


from matplotlib import pyplot as plt

pdread = pd.read_csv('output.csv', sep=';')
fig = plt.figure(figsize=(12,8))

list_diff = diff_weigth.tolist()

def certainValue(type) :
    value =[]
    if (type =="Weigth") :
        for i in range(1, 578):
            value.append(pdread[type][i])
        return value
    if (type == "Steps") :
        return pdread.loc[pdread['Day'] == 'W'].Steps.apply(lambda x: float(x))

def difference(type) :
    pdtype = pdread[type]
    diff = []
    if (type == "Weigth") :
        for i in range(2, 577):
            diff.append(pdtype[i + 1] - pdtype[i])
    return diff

def ARIMAtest(list, AR, MA):
    model = ARIMA(list, order = (AR,1,MA)) #Weight : order = (2,1,1), stepsWednesday : (1,0,1)
    results = model.fit()
    return results

def ACF_PACFfigure(fig, list):
    ax1 = fig.add_subplot(211)
    fig = sm.graphics.tsa.plot_acf(list, lags=20, ax = ax1)
    ax2 = fig.add_subplot(212)
    fig = sm.graphics.tsa.plot_pacf(list, lags=20, ax= ax2)
    plt.show()

def ARIMAresid(ARIMAsummary):
    return ARIMAsummary.resid


def ARIMAPredict(ARIMAResult, numberDaysPredict):
    predict = ARIMAResult.predict(start=80, end=90+numberDaysPredict)
    ARIMAResult.plot_predict(dynamic=False, start=80, end= 90+numberDaysPredict)
    plt.title("Prédiction de la différence du poids") #Changer le titre
    plt.show()
    return predict

def BoxPierce(list):
    print('Résultat de BoxPierce ', sm.stats.diagnostic.acorr_ljungbox(list, return_df=True, lags=1))

def predictWeigth():
    ACF_PACFfigure(fig, list_diff) #"On a MA(1), MA(2) et AR(1), AR(2), AR(3) et AR(4)"
    #print(list_diff)
    #BoxPierce(certainValue("Weigth"))
    x = ARIMAtest(list_diff, 1, 1)
    print(x.summary())
    print(ARIMAPredict(x, 1))

def predictSteps():
    ACF_PACFfigure(fig, certainValue("Steps")) # On prend MA(0), AR(0)
    print(certainValue("Steps")) #changer la valeur du jour au niveau de la fonction
    BoxPierce(certainValue("Steps"))
    x = ARIMAtest(certainValue("Steps"), 1,1)
    print(x.summary())
    print(ARIMAPredict(x,1))

def run() :
    #predictSteps()
    predictWeigth()

run()